In [ ]:
import pandas as pd
import paramiko
import os
from row64tools import ramdb

In [ ]:
localfile = r"C:\Users\mikha\LapAnalysisKTM.ramdb"
remote_path = "/var/www/ramdb/live/RAMDB.MikhailData/MikhailData/LapAnalysis.ramdb"
# Ubuntu server credentials
hostname = "192.168.1.20"   # Replace with your Ubuntu server's local IP
port = 22
username = "row64"   # Ubuntu login username
password = "temp7"   # Or use SSH key auth
total_records = 3_299_165      # total records to simulate
batch_size = 300_000           # number of rows per batch


In [ ]:
transport = paramiko.Transport((hostname, port))
transport.connect(username=username, password=password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [ ]:
for batch_start in range(0, total_records, batch_size):
    batch_end = min(batch_start + batch_size, total_records)
    
    # Create batch dataframe
    df_batch = pd.DataFrame({
        "ID": range(batch_start + 1, batch_end + 1),
        "Value": [f"Record_{i}" for i in range(batch_start + 1, batch_end + 1)]
    })
    
    # Append to local CSV (create file if it doesn't exist)
    if os.path.exists(local_file):
        df_batch.to_csv(local_file, mode='a', header=False, index=False)
    else:
        df_batch.to_csv(local_file, index=False)
    
    # Upload the growing file to Ubuntu
    sftp.put(local_file, remote_file)
    print(f"✅ Appended rows {batch_start+1}-{batch_end} and uploaded.")

    # Optional delay to simulate real-time streaming
    time.sleep(2)
